<a href="https://colab.research.google.com/github/tangQAQ/LSTM_training/blob/main/LSTM(Pct).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [83]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, TimeDistributed
from google.colab import drive
import matplotlib.pyplot as plt
from keras.optimizers import Adam
import warnings 

warnings.filterwarnings('ignore')

drive.mount('/content/gdrive')
train_data = pd.read_csv("/content/gdrive/MyDrive/GSPC_train.csv", )
train_data = train_data.drop(['Date'], axis= 1)
train_data = train_data.apply(lambda x: pd.to_numeric(x.str.replace(',',''), errors='coerce')).fillna(-1).astype('float32')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [84]:
train_data['Pct_change'] = train_data['Close'].pct_change()
train_data.drop(train_data.index[0], inplace=True)
train_data

,Open,High,Low,Close,Adj Close,Volume,Pct_change
1,17.719999,17.719999,17.719999,17.719999,17.719999,-1.000000e+00,-0.002252
2,17.549999,17.549999,17.549999,17.549999,17.549999,-1.000000e+00,-0.009594
3,17.660000,17.660000,17.660000,17.660000,17.660000,-1.000000e+00,0.006268
4,17.500000,17.500000,17.500000,17.500000,17.500000,-1.000000e+00,-0.009060
5,17.370001,17.370001,17.370001,17.370001,17.370001,-1.000000e+00,-0.007429
...,...,...,...,...,...,...,...
23678,4494.169922,4503.939941,4450.040039,4481.149902,4481.149902,4.986830e+09,-0.009717
23679,4474.649902,4521.160156,4450.299805,4500.209961,4500.209961,4.821490e+09,0.004253
23680,4494.149902,4520.410156,4474.600098,4488.279785,4488.279785,4.083200e+09,-0.002651
23681,4462.640137,4464.350098,4408.379883,4412.529785,4412.529785,4.266290e+09,-0.016877


In [85]:
def replace_values(x):
  if x < -10:
    return -11
  elif x > 10:
    return 11
  elif x >= 0 and x <=1 :
    return 1
  elif x < 0 and x >= -1:
    return -1
  else:
    return x

In [112]:
train_data['feature_cheak'] = train_data['Pct_change'].round(2)*100
train_data['feature'] = train_data['feature_cheak'].apply(replace_values)
train_data

,Open,High,Low,Close,Adj Close,Volume,Pct_change,feature_cheak,feature1,feature
1,17.719999,17.719999,17.719999,17.719999,17.719999,-1.000000e+00,-0.002252,-0.0,1.0,1.0
2,17.549999,17.549999,17.549999,17.549999,17.549999,-1.000000e+00,-0.009594,-1.0,-1.0,-1.0
3,17.660000,17.660000,17.660000,17.660000,17.660000,-1.000000e+00,0.006268,1.0,1.0,1.0
4,17.500000,17.500000,17.500000,17.500000,17.500000,-1.000000e+00,-0.009060,-1.0,-1.0,-1.0
5,17.370001,17.370001,17.370001,17.370001,17.370001,-1.000000e+00,-0.007429,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...
23678,4494.169922,4503.939941,4450.040039,4481.149902,4481.149902,4.986830e+09,-0.009717,-1.0,-1.0,-1.0
23679,4474.649902,4521.160156,4450.299805,4500.209961,4500.209961,4.821490e+09,0.004253,0.0,1.0,1.0
23680,4494.149902,4520.410156,4474.600098,4488.279785,4488.279785,4.083200e+09,-0.002651,-0.0,1.0,1.0
23681,4462.640137,4464.350098,4408.379883,4412.529785,4412.529785,4.266290e+09,-0.016877,-2.0,-2.0,-2.0


In [113]:
df = pd.DataFrame()
df = train_data['feature']
df = df[(df >= -3) & (df <= 2)]
df

1        1.0
2       -1.0
3        1.0
4       -1.0
5       -1.0
        ... 
23678   -1.0
23679    1.0
23680    1.0
23681   -2.0
23682    1.0
Name: feature, Length: 22996, dtype: float64

In [114]:
unique, counts = np.unique(df, return_counts=True)

for val, count in zip(unique, counts):
    print(f"{val}: {count}")

-3.0: 308
-2.0: 954
-1.0: 4067
1.0: 16680
2.0: 987


In [115]:
std = np.std(train_data['feature'])
mean = np.mean(train_data['feature'])
std, mean

(1.3051101494461512, 0.5265180305717423)

In [116]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(sparse=False)
one_hot = enc.fit_transform(train_data['feature'].values.reshape(-1,1))
one_hot.shape

(23682, 22)

In [117]:
oh = pd.DataFrame(one_hot)
oh

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23677,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23678,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23679,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23680,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [118]:
def data_split(data, slice_size):
  size = int(len(data) * slice_size)
  train = data[:size]
  val = data[size:]
  return train, val

In [119]:
def create_sequences(data, sequences):
    X = []
    y = []
    for i in range(sequences, len(data)):
        X.append(data.iloc[i-sequences:i, :])
        y.append(data.iloc[i, 0])
    X = np.array(X)
    y = np.array(y)
    return X, y

In [120]:
train, val = data_split(oh, 0.8)
train.shape, val.shape

((18945, 22), (4737, 22))

In [121]:
sequences = 15

X_train, y_train = create_sequences(oh, sequences)
X_val, y_val = create_sequences(oh, sequences)
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((23667, 15, 22), (23667,), (23667, 15, 22), (23667,))

In [162]:
model = Sequential()
model.add(LSTM(units=64, activation='relu', return_sequences=False, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(units=X_train.shape[2], activation=''))

model.compile(optimizer='Adam', loss='mse')

model.fit(X_train, y_train, validation_data=(X_val, y_val),epochs=30, batch_size=64)

Epoch 1/30
370/370 [==============================] - 61s 130ms/step - loss: 6.1109e-04 - val_loss: 1.2676e-04
Epoch 2/30
370/370 [==============================] - 49s 133ms/step - loss: 1.2676e-04 - val_loss: 1.2676e-04
Epoch 3/30
370/370 [==============================] - ETA: 0s - loss: 1.2676e-04

KeyboardInterrupt: ignored

In [ ]:
model.evaluate(X_train, y_train, batch_size= 32)

In [ ]:
test_df = pd.read_csv("/content/gdrive/MyDrive/GSPC_test.csv")
days = 15

test_data = test_df.copy()
test_data = test_data.drop(['Date'], axis= 1)
test_data_clean = test_data.apply(lambda x: pd.to_numeric(x.str.replace(',',''), errors='coerce')).fillna(-1).astype('float32')
#test_data_clean = test_data_clean.iloc[:sequences+2, :]
test_data_clean['Pct_change'] = test_data_clean['Close'].pct_change()
test_data_clean.drop(test_data.index[0], inplace=True)

In [ ]:
test_data_clean['feature_cheak'] = test_data_clean['Pct_change'].round(2)*100
test_data_clean['feature'] = test_data_clean['feature_cheak'].apply(replace_values)
test_data_clean

In [ ]:
toh = enc.transform(test_data_clean['feature'].values.reshape(-1, 1))
toh = pd.DataFrame(toh)
toh.shape

In [ ]:
X_test, y_test = create_sequences(toh, sequences)
test_pred = model.predict(X_test)
inverse = enc.inverse_transform(test_pred)
preds = np.array(inverse)


In [ ]:
preds.shape

In [ ]:
model.evaluate(X_test, y_test, batch_size= 32)

In [ ]:
from numpy.ma import left_shift
import matplotlib.ticker as ticker


limit = sequences + days
Actual_Close = test_data_clean['feature'].drop(sequences)

result = preds

plt.figure(figsize=(12, 8))
plt.title('Actual vs. Predicted Closing Prices')
plt.xlabel('Time line')
plt.ylabel('Closing Price')
plt.plot(result, label='Predicted Close', color='b', linestyle='--')
plt.plot(Actual_Close, label='Actual Close', color='r')
plt.xlim([0,days])
#plt.ylim([max(Actual_Close), min(result)])
ax= plt.gca()
#ax.xaxis.set_major_locator(ticker.MultipleLocator(100))
#ax.yaxis.set_major_locator(ticker.MultipleLocator(100))
plt.legend()
plt.show()